In [41]:
import pandas as pd
import numpy as np
from pandas.io.html import read_html

In [63]:
#Extracting the data from the wiki
page= 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050.' #permanent link to a specific revision to prevent issues from changes in formating
table= read_html(page, attrs={'class':'wikitable'}) #the class is obtainable by inspecting the table on the wiki
table=table[0]
table.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [66]:
#processing the data: removing any entry without a borough assigned
table=table[table['Borough']!="Not assigned"]
table=table.reset_index(drop=True)
table.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [72]:
#processing the data: setting any entry without a neighbourhood assigned to have it set to its borough
table['Neighbourhood']=table['Neighbourhood'].replace('Not assigned', table['Borough'])
table.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [80]:
#processing the data:grouping neighbourhoods by postcode
table=table.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
table.head(20)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [79]:
table.shape

(103, 3)